In [8]:
import numpy as np
#
def mr_svd(M, m, n):
    # Direct SVD decomposition multi-resolution
    # Input: M     - (m x n) matrix to SVD decomposition
    #        m x n - Image dimension 
    # Output: Return list Y decomposition and matrix U of the SVD decomposition
    #         Where are TLL, TLH, THL, and THH into a list Y
    # 
    # Set multi-resolution two level
    m = int(m/2)
    n = int(n/2)
    # Set md to two level SVD decomposition IM.LL, IM.LH, IM.HL, and IM.HH
    # Obs: Each decomposition level split the initial image into 4 matrix
    md = 4
    # Resize M into matrix A[4, m * n]
    A = np.zeros((md, m * n))
    for j in range(n):
        for i in range(m):
            for l in range(2):
                for k in range(2):
                    A[k + l * 2, i + j * m] = M[i * 2 + k, j * 2 + l]
    #
    # Calculate SVD decomposition to A
    U, S, V = np.linalg.svd(A, full_matrices=True)
    UT =  U.transpose()
    T = UT @ A
    # Set each line of T into a vector TLL, TLH, THL, and THH
    TLL = np.zeros((m, n))
    TLH = np.zeros((m, n))
    THL = np.zeros((m, n))
    THH = np.zeros((m, n))
    for j in range(n):
        for i in range(m):
            TLL[i, j] = T[0, i + j * m]
            TLH[i, j] = T[1, i + j * m]
            THL[i, j] = T[2, i + j * m]
            THH[i, j] = T[3, i + j * m]
    #
    # Put TLL, TLH, THL, and THH into a list Y
    Y = []
    Y.append(TLL)
    Y.append(TLH)
    Y.append(THL)
    Y.append(THH)
    # Return Y decomposition and matrix U of the SVD decomposition
    return Y, U
#
def mr_isvd(Y, U):
    # Inverse SVD decomposition multi-resolution
    # Input: List Y with coeficients and matrix U fusion to SVD inverse decomposition
    #         Where TLL, TLH, THL, and THH are into a list Y
    # Output: Image fusion
    # Define dimension
    dim = Y[0].shape
    m = dim[0]
    n = dim[1]
    mn = dim[0] * dim[1]
    # Put list Y into matrix T[4, m * n]
    # Obs: Each decomposition level split the initial image into 4 matrix
    #
    T = np.zeros((4, mn))
    for j in range(n):
        for i in range(m):
            T[0, i + j * m] = Y[0][i][j]
            T[1, i + j * m] = Y[1][i][j]
            T[2, i + j * m] = Y[2][i][j]
            T[3, i + j * m] = Y[3][i][j]
    #
    # Inverse SVD
    A = U @ T
    # Put A into matrix M
    M = np.zeros((2 * m, 2 * n))
    for j in range(n):
        for i in range(m):
            for l in range(2):
                for k in range(2):
                    M[i * 2 + k, j * 2 + l] = A[k + l * 2, i + j * m]
    # Return the image M
    return M
#
def fus_svd(E, m, n, nc):
    # SVD multi-resolution Fusion
    # Input: E     - (m x n x nc) Data with one image per channel
    #        m x n - Image dimension
    #        nc    - Channels number      
    # Output: F - Image fusion
    # Realize the SVD FUSION
    XC = []
    UC = []
    # Calculate the SVD methods for each image (channel)
    # Storage into two list
    for c in range(nc):
        X, U = mr_svd(E[:, :, c], m, n)
        XC.append(X)
        UC.append(U)
    #
    # Set de dimension
    mr = int(m / 2)
    nr = int(n / 2)
    SOMA = np.zeros((mr, nr))
    XLL  = np.zeros((mr, nr))
    # Calculate the average in alls decompositions X.LL (among channel)
    for c in range(nc):
        SOMA = SOMA + XC[c][0]
    XLL = SOMA / nc
    #
    XF = []
    XF.append(XLL)
    #
    # Obs: Each decomposition level split the initial image into 4 matrix
    nd = 4
    # Calculate the maximum in alls decompositions X.LH, X.HL, and X.HH (among channel)
    for c in range(1, nd):
        D = np.maximum(XC[0][c], XC[1][c])>= 0
        # Element-wise multiplication, and rule to fusion
        XA = D * XC[0][c] + ~D * XC[1][c]
        D = np.maximum(XA, XC[2][c])>= 0
        # Element-wise multiplication, and rule to fusion
        COEF = D * XA + ~D * XC[2][c]
        XF.append(COEF)
    #
    # Rule fusion to matriz list UC
    SOMA1 = np.zeros((4, 4))
    UF    = np.zeros((4, 4))
    for c in range(nc):
        SOMA1 = SOMA1 + UC[c]
    UF = SOMA1 / nc
    IF = mr_isvd(XF, UF)
    return IF
#
# Programa principal para validacao do codigo
#
m = 16
n = 16
nc = 3
MAT = np.zeros((m, n))
for j in range(n):
    for i in range(m):
        #MAT[i, j] = 1
        #MAT[i, j] = (i + 1) + (j + 1)
        MAT[i, j] = 1 / ((i + 1) + (j + 1))
#
IMTES = np.zeros((m, n, nc))
IMTES[:,:, 0] = MAT
IMTES[:,:, 1] = MAT
IMTES[:,:, 2] = MAT
TESTE = fus_svd(IMTES, m, n, nc)

In [9]:
np.abs(MAT - TESTE)

array([[1.66533454e-16, 0.00000000e+00, 1.11022302e-16, 2.77555756e-17,
        8.32667268e-17, 2.77555756e-17, 6.93889390e-17, 1.38777878e-17,
        6.93889390e-17, 1.38777878e-17, 4.16333634e-17, 0.00000000e+00,
        5.55111512e-17, 1.38777878e-17, 2.77555756e-17, 0.00000000e+00],
       [1.11022302e-16, 5.55111512e-17, 2.77555756e-17, 0.00000000e+00,
        2.77555756e-17, 0.00000000e+00, 1.38777878e-17, 1.38777878e-17,
        1.38777878e-17, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.38777878e-17, 6.93889390e-18, 0.00000000e+00, 0.00000000e+00],
       [1.11022302e-16, 2.77555756e-17, 8.32667268e-17, 2.77555756e-17,
        6.93889390e-17, 1.38777878e-17, 6.93889390e-17, 1.38777878e-17,
        4.16333634e-17, 0.00000000e+00, 5.55111512e-17, 1.38777878e-17,
        2.77555756e-17, 0.00000000e+00, 3.46944695e-17, 0.00000000e+00],
       [2.77555756e-17, 0.00000000e+00, 2.77555756e-17, 0.00000000e+00,
        1.38777878e-17, 1.38777878e-17, 1.38777878e-17, 0.000